In [ ]:
import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.multioutput import MultiOutputRegressor
from lightgbm import LGBMRegressor
import xgboost as xgb
from textblob import TextBlob
import spacy
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from collections import Counter
from sklearn.linear_model import Ridge
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import Normalizer
from tqdm import tqdm
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from string import punctuation
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn import preprocessing
trans = preprocessing.MinMaxScaler()
from collections import Counter
from sklearn.model_selection import train_test_split

In [ ]:
# Importar los datos de drive
from google.colab import drive
from google.colab import files
import io
drive.mount('/content/drive')

train = pd.read_csv('/content/drive/MyDrive/Tesis/train_modificado.csv')
test = pd.read_csv('/content/drive/MyDrive/Tesis/test_modificado.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **Normalización**

In [ ]:
#Toma las características extraídas
variables = [var for var in train.columns if var not in ['full_text', 'text_id', 'cohesion','syntax','vocabulary','phraseology','grammar','conventions']]

In [ ]:
#Lista de variables
variables

['Unnamed: 0',
 'word_count',
 'diff',
 'n_capital',
 'n_punct',
 'spelling_errors',
 'n_unique',
 'n_unique_n_stop',
 'n_n_word',
 'noun_phrase_count',
 'PRON',
 'VERB',
 'SCONJ',
 'NOUN',
 'AUX',
 'ADP',
 'PUNCT',
 'PART',
 'CCONJ',
 'ADV',
 'DET',
 'ADJ',
 'SPACE',
 'PROPN',
 'NUM',
 'INTJ',
 'SYM',
 'X',
 'polarity',
 'subjectivity',
 'spell_score',
 'av_sent_len',
 'max_sent_len',
 'min_sent_len',
 'med_sent_len',
 'std_sent_len',
 'num_sent',
 'compound',
 'negative',
 'positive',
 'neutral',
 'char_len']

In [ ]:
#Normaliza las variables para el conjunto de entrenamiento
X_train_custom=trans.fit_transform(train[variables].values)
X_train_custom.shape

(3128, 42)

In [ ]:
#Normaliza las variables para el conjunto de prueba
X_test_custom=trans.fit_transform(test[variables].values)
X_test_custom.shape

(783, 42)

In [ ]:
#Vectorización
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(train['full_text'])
X_test = vectorizer.transform(test['full_text'])

In [ ]:
X_train=np.asarray(X_train.todense())
X_test=np.asarray(X_test.todense())

In [ ]:
#Targets
y_vars = ['cohesion','syntax','vocabulary','phraseology','grammar','conventions']

In [ ]:
y_train = train[y_vars]
y_test = test[y_vars]

# **Métrica**

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

def RMSE(y, y_hat):
    return np.sqrt(mean_squared_error(y,y_hat))

def MCRMSE(y_df, y_hat_df):
    size = y_df.shape[1]
    mcrmse = 0
    for i in range(size):
        mcrmse+=RMSE(y_df.iloc[:, i], y_hat_df[:, i])
    return mcrmse/size

#**Modelos Lineales**

## 1. LGBM

**LGBM Multi Output Regressor para X_train**




In [ ]:
params_lgb = {
    "n_estimators": 1000,
    "verbose": -1
}

In [ ]:
model = MultiOutputRegressor(LGBMRegressor(**params_lgb))
model.fit(X_train, y_train)

MultiOutputRegressor(estimator=LGBMRegressor(n_estimators=1000, verbose=-1))

In [ ]:
y_pred = model.predict(X_test)
print(MCRMSE(y_test,y_pred))
print(mean_absolute_error(y_test, y_pred))

0.5537921813304042
0.4444616758999591


**LGBM Multi Output Regressor para X_train_custom**

In [ ]:
params_lgb = {
    "n_estimators": 1000,
    "verbose": -1
}

In [ ]:
model = MultiOutputRegressor(LGBMRegressor(**params_lgb))
model.fit(X_train_custom, y_train)

MultiOutputRegressor(estimator=LGBMRegressor(n_estimators=1000, verbose=-1))

In [ ]:
y_pred = model.predict(X_test_custom)
print(MCRMSE(y_test,y_pred))
print(mean_absolute_error(y_test, y_pred))

0.6553136148931762
0.5237002245680429


**LGBM Cross Validated para X_train**

In [ ]:
params_lgb = {'subsample_freq': 0, 'reg_lambda': 61, 'reg_alpha': 21, 'num_leaves': 85, 'num_iterations': 110, 'n_estimators': 4900, 'min_data_in_leaf': 11, 'max_depth': 30, 'max_bin': 100, 'learning_rate': 0.1, 'importance_type': 'gain', 'feature_fraction': 0.2, 'boosting_type': 'gbdt', 'bagging_fraction': 0.3}

In [ ]:
model = MultiOutputRegressor(LGBMRegressor(**params_lgb))
model.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] min_data_in_leaf is set=11, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=11
[LightGBM] [Warning] feature_fraction is set=0.2, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2
[LightGBM] [Warning] bagging_fraction is set=0.3, subsample=1.0 will be ignored. Current value: bagging_fraction=0.3
[LightGBM] [Warning] min_data_in_leaf is set=11, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=11
[LightGBM] [Warning] feature_fraction is set=0.2, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2
[LightGBM] [Warning] bagging_fraction is set=0.3, subsample=1.0 will be ignored. Current value: bagging_fraction=0.3
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.028916 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 78767
[LightGBM] [Info] Number of data points in the train set: 3128, number of use

/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] min_data_in_leaf is set=11, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=11
[LightGBM] [Warning] feature_fraction is set=0.2, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2
[LightGBM] [Warning] bagging_fraction is set=0.3, subsample=1.0 will be ignored. Current value: bagging_fraction=0.3
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.030763 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 78767
[LightGBM] [Info] Number of data points in the train set: 3128, number of used features: 2809
[LightGBM] [Info] Start training from score 3.031330
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain

/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


...

/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] min_data_in_leaf is set=11, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=11
[LightGBM] [Warning] feature_fraction is set=0.2, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2
[LightGBM] [Warning] bagging_fraction is set=0.3, subsample=1.0 will be ignored. Current value: bagging_fraction=0.3
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.044513 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 78767
[LightGBM] [Info] Number of data points in the train set: 3128, number of used features: 2809
[LightGBM] [Info] Start training from score 3.031330
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain

/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] min_data_in_leaf is set=11, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=11
[LightGBM] [Warning] feature_fraction is set=0.2, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2
[LightGBM] [Warning] bagging_fraction is set=0.3, subsample=1.0 will be ignored. Current value: bagging_fraction=0.3
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.045034 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 78767
[LightGBM] [Info] Number of data points in the train set: 3128, number of used features: 2809
[LightGBM] [Info] Start training from score 3.082001
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain

MultiOutputRegressor(estimator=LGBMRegressor(bagging_fraction=0.3,
                                             feature_fraction=0.2,
                                             importance_type='gain',
                                             max_bin=100, max_depth=30,
                                             min_data_in_leaf=11,
                                             n_estimators=4900,
                                             num_iterations=110, num_leaves=85,
                                             reg_alpha=21, reg_lambda=61))

In [ ]:
y_pred = model.predict(X_test)
print(MCRMSE(y_test,y_pred))
print(mean_absolute_error(y_test, y_pred))

[LightGBM] [Warning] min_data_in_leaf is set=11, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=11
[LightGBM] [Warning] feature_fraction is set=0.2, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2
[LightGBM] [Warning] bagging_fraction is set=0.3, subsample=1.0 will be ignored. Current value: bagging_fraction=0.3
[LightGBM] [Warning] min_data_in_leaf is set=11, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=11
[LightGBM] [Warning] feature_fraction is set=0.2, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2
[LightGBM] [Warning] bagging_fraction is set=0.3, subsample=1.0 will be ignored. Current value: bagging_fraction=0.3
[LightGBM] [Warning] min_data_in_leaf is set=11, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=11
[LightGBM] [Warning] feature_fraction is set=0.2, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2
[LightGBM] [Warning] bagging

**LGBM Cross Validated para X_train_custome**

In [ ]:
params_lgb = {'subsample_freq': 0, 'reg_lambda': 61, 'reg_alpha': 21, 'num_leaves': 85, 'num_iterations': 110, 'n_estimators': 4900, 'min_data_in_leaf': 11, 'max_depth': 30, 'max_bin': 100, 'learning_rate': 0.1, 'importance_type': 'gain', 'feature_fraction': 0.2, 'boosting_type': 'gbdt', 'bagging_fraction': 0.3}

In [ ]:
model = MultiOutputRegressor(LGBMRegressor(**params_lgb))
model.fit(X_train_custom, y_train)

/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] min_data_in_leaf is set=11, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=11
[LightGBM] [Warning] feature_fraction is set=0.2, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2
[LightGBM] [Warning] bagging_fraction is set=0.3, subsample=1.0 will be ignored. Current value: bagging_fraction=0.3
[LightGBM] [Warning] min_data_in_leaf is set=11, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=11
[LightGBM] [Warning] feature_fraction is set=0.2, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2
[LightGBM] [Warning] bagging_fraction is set=0.3, subsample=1.0 will be ignored. Current value: bagging_fraction=0.3
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000627 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3125
[LightGBM] [Info] Number of data points in the train set: 3128, number of used

/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


MultiOutputRegressor(estimator=LGBMRegressor(bagging_fraction=0.3,
                                             feature_fraction=0.2,
                                             importance_type='gain',
                                             max_bin=100, max_depth=30,
                                             min_data_in_leaf=11,
                                             n_estimators=4900,
                                             num_iterations=110, num_leaves=85,
                                             reg_alpha=21, reg_lambda=61))

In [ ]:
y_pred = model.predict(X_test_custom)
print(MCRMSE(y_test,y_pred))
print(mean_absolute_error(y_test, y_pred))

[LightGBM] [Warning] min_data_in_leaf is set=11, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=11
[LightGBM] [Warning] feature_fraction is set=0.2, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2
[LightGBM] [Warning] bagging_fraction is set=0.3, subsample=1.0 will be ignored. Current value: bagging_fraction=0.3
[LightGBM] [Warning] min_data_in_leaf is set=11, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=11
[LightGBM] [Warning] feature_fraction is set=0.2, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2
[LightGBM] [Warning] bagging_fraction is set=0.3, subsample=1.0 will be ignored. Current value: bagging_fraction=0.3
[LightGBM] [Warning] min_data_in_leaf is set=11, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=11
[LightGBM] [Warning] feature_fraction is set=0.2, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2
[LightGBM] [Warning] bagging

## 2. Regresión Lineal

In [ ]:
from sklearn.linear_model import LinearRegression

**Con X_train**

In [ ]:
params_reg = {
    "n_estimators": 1000,
    "verbose": -1
}

In [ ]:
model = MultiOutputRegressor(LinearRegression())
model.fit(X_train, y_train)

MultiOutputRegressor(estimator=LinearRegression())

In [ ]:
y_pred = model.predict(X_test)
print(MCRMSE(y_test,y_pred))
print(mean_absolute_error(y_test, y_pred))

0.725830013486247
0.579226035218831


**Con X_train_custom**

In [ ]:
params_reg = {
    "n_estimators": 1000,
    "verbose": -1
}

In [ ]:
model = MultiOutputRegressor(LinearRegression())
model.fit(X_train_custom, y_train)

MultiOutputRegressor(estimator=LinearRegression())

In [ ]:
y_pred = model.predict(X_test_custom)
print(MCRMSE(y_test,y_pred))
print(mean_absolute_error(y_test, y_pred))

0.6993832106685955
0.5548402653568189


## 3. XGBoost

In [ ]:
import xgboost as xgb

**XGBoost para X_train**

In [ ]:
params_xgb = {'n_estimators': 1000, 'random_state':0}

In [37]:
model = MultiOutputRegressor(xgb.XGBRegressor(
        **params_xgb,
        objective='reg:squarederror'))
model.fit(X_train, y_train)

MultiOutputRegressor(estimator=XGBRegressor(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            feature_types=None, gamma=None,
                                            grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=None, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=None,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=1000, n_jobs=None,
                                            num_parallel_tree=None,
                                            random_state=0, ...))

In [38]:
y_pred = model.predict(X_test)
print(RMSE(y_test,y_pred))
print(mean_absolute_error(y_test, y_pred))

0.5829136795459533
0.4654132361968257


**XGBoost para X_train_custome**

In [39]:
params_xgb = {'n_estimators': 1000, 'random_state':0}

In [40]:
model = MultiOutputRegressor(xgb.XGBRegressor(
        **params_xgb,
        objective='reg:squarederror'))
model.fit(X_train_custom, y_train)

MultiOutputRegressor(estimator=XGBRegressor(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            feature_types=None, gamma=None,
                                            grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=None, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=None,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=1000, n_jobs=None,
                                            num_parallel_tree=None,
                                            random_state=0, ...))

In [41]:
y_pred = model.predict(X_test_custom)
print(RMSE(y_test,y_pred))
print(mean_absolute_error(y_test, y_pred))

0.6716569759393711
0.5357783622770119


**XGBoost Cross Validated para X_train**

In [42]:
params_xgb_cv =  {'subsample': 0.8, 'reg_alpha': 0, 'n_estimators': 740, 'min_child_weight': 12, 'max_depth': 6, 'lambda': 5, 'gamma': 0.3, 'eval_metric': 'rmse', 'eta': 0.03, 'colsample_bytree': 0.85, 'booster': 'gbtree'}

In [43]:
model = MultiOutputRegressor(xgb.XGBRegressor(
        **params_xgb,
        objective='reg:squarederror'))
model.fit(X_train, y_train)

MultiOutputRegressor(estimator=XGBRegressor(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            feature_types=None, gamma=None,
                                            grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=None, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=None,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=1000, n_jobs=None,
                                            num_parallel_tree=None,
                                            random_state=0, ...))

In [44]:
y_pred = model.predict(X_test)
print(RMSE(y_test,y_pred))
print(mean_absolute_error(y_test, y_pred))

0.5829136795459533
0.4654132361968257


**XGBoost Cross Validated para X_train_custome**

In [45]:
params_xgb_cv =  {'subsample': 0.8, 'reg_alpha': 0, 'n_estimators': 740, 'min_child_weight': 12, 'max_depth': 6, 'lambda': 5, 'gamma': 0.3, 'eval_metric': 'rmse', 'eta': 0.03, 'colsample_bytree': 0.85, 'booster': 'gbtree'}

In [46]:
model = MultiOutputRegressor(xgb.XGBRegressor(
        **params_xgb,
        objective='reg:squarederror'))
model.fit(X_train_custom, y_train)

MultiOutputRegressor(estimator=XGBRegressor(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            feature_types=None, gamma=None,
                                            grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=None, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=None,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=1000, n_jobs=None,
                                            num_parallel_tree=None,
                                            random_state=0, ...))

In [47]:
y_pred = model.predict(X_test_custom)
print(RMSE(y_test,y_pred))
print(mean_absolute_error(y_test, y_pred))

0.6716569759393711
0.5357783622770119
